## Calculating Distance using Lat longs

Given 2 points with their Latitude and Longitude coordinates, the Haversine Formula calculates the straight-line distance in meters, assuming that Earth is a sphere.



In [12]:
san_francisco = (37.7749, -122.4194)
new_york = (40.661, -73.944) #lat , long in degree decimal

In [13]:
import math 

In [14]:
def haversine_distance(origin, destination):
  lat1, lon1 = origin
  lat2, lon2 = destination
  radius = 6371000
  dlat = math.radians(lat2-lat1)
  dlon = math.radians(lon2-lon1)
  a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
    * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
  distance = radius * c
  return distance

In [15]:
distance = haversine_distance(san_francisco, new_york)
print(distance/1000, 'km')

4135.374617164737 km


## Now calculating Distance Using Geopy package

- `distance.great_circle()`: Calculates the distance on a great circle using haversine formula
- `distance.geodesic()`: Calculates the distance using a chosen ellipsoid using vincenty's formula

In [16]:
from geopy import distance

san_francisco = (37.7749, -122.4194)
new_york = (40.661, -73.944)

straight_line_distance = distance.great_circle(san_francisco, new_york)
ellipsoid_distance = distance.geodesic(san_francisco, new_york, ellipsoid='WGS-84')

print(straight_line_distance, ellipsoid_distance)

4135.3804590061345 km 4145.446977549561 km


# Geocoding

Geocoding is the process of converting a human-readable location description,
such as a street address or the name of a place, into geographic coordinates
(latitude and longitude) that identify a specific point on Earth's surface.
This is commonly done to place markers on a map, center a map on a location, or for logistical purposes like deliveries. 
The reverse process, converting coordinates into an address, is called reverse geocoding

In [17]:
from geopy.geocoders import Nominatim

# Initialize the geocoder
geolocator = Nominatim(user_agent="my-geocoding-app")

# Geocode an address
location = geolocator.geocode("Bandarulanka")
if location:
    print(f"Address: {location.address}")
    print(f"Latitude: {location.latitude}, Longitude: {location.longitude}")

# Reverse geocode coordinates
coordinates = location.latitude , location.longitude
reverse_location = geolocator.reverse(coordinates)
if reverse_location:
    print(f"Reverse Geocoded Address: {reverse_location.address}")

Address: Bandarulanka, Idigagarru, Amalapuram, Dr. B. R. Ambedkar Konaseema, Andhra Pradesh, 533221, India
Latitude: 16.5868087, Longitude: 81.9692743
Reverse Geocoded Address: Amalapuram - Bobbarlanka Road, Bandarulanka, Amalapuram, Dr. B. R. Ambedkar Konaseema, Andhra Pradesh, 533221, India


### Note:- The geopy itself not provide any Geocoding server. Geopy it self depends on the another API. which provides the Geocoding and reverse Geocoding. EX:- Googlemaoapi's , Openstreetmap api's etc

## Using Web APIs

For mapping and spatial analysis, being able to use APIs is critical. For the longest time, Google Maps API was the most popular API on the web. APIs allow you to query web servers and get results without downloading data or running computation on your machine.

Common use cases for using APIs for spatial analysis are

- Getting directions / routing
- Route optimization
- Geocoding
- Downloading data
- Getting real-time weather data
- ...

The provide of such APIs have many ways to implement an API. There are standards such as REST, SOAP, GraphQL etc. *REST* is the most populat standard for web APIs, and for geospatial APIs. REST APIs are used over HTTP and thus called web APIs.


## Understanding JSON and GeoJSON

JSON stands for **J**ava**S**cript **O**bject **N**otation. It is a format for storing and transporting data, and is the de-facto standard for data exchanged by APIs. GeoJSON is an extension of the JSON format that is commonly used to represent spatial data.

Python has a built-in `json` module that has methods for reading json data and converting it to Python objects, and vice-versa. In this example, we are using the `requests` module for querying the API which conveniently does the conversion for us. But it is useful to learn the basics of working with JSON in Python.

In [18]:
geojson_string = '''
{
  "type": "FeatureCollection",
  "features": [
    {"type": "Feature",
      "properties": {"name": "San Francisco"},
      "geometry": {"type": "Point", "coordinates": [-121.5687, 37.7739]}
    }
  ]
}
'''
print(geojson_string)


{
  "type": "FeatureCollection",
  "features": [
    {"type": "Feature",
      "properties": {"name": "San Francisco"},
      "geometry": {"type": "Point", "coordinates": [-121.5687, 37.7739]}
    }
  ]
}



To convert a JSON string to a Python object (i.e. parsing JSON), we can use the `json.loads()` method.

In [19]:
import json

data = json.loads(geojson_string)
print(type(data))
print(data)

<class 'dict'>
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'San Francisco'}, 'geometry': {'type': 'Point', 'coordinates': [-121.5687, 37.7739]}}]}


Now that we have *parsed* the GeoJSON string and have a Python object, we can extract infromation from it. The data is stored in a *FeatureCollection* - which is a list of *features*. In our example, we have just 1 feature inside the feature collection, so we can access it by using index **0**.

In [20]:
city_data = data['features'][0]
print(city_data)

{'type': 'Feature', 'properties': {'name': 'San Francisco'}, 'geometry': {'type': 'Point', 'coordinates': [-121.5687, 37.7739]}}


In [21]:
city_name = city_data['properties']['name']
city_coordinates = city_data['geometry']['coordinates']
print(city_name, city_coordinates)

San Francisco [-121.5687, 37.7739]


# The Request Module

To query a server, we send a **GET** request with some parameters and the server sends a response back. The `requests` module allows you to send HTTP requests and parse the responses using Python.

The response contains the data received from the server. It contains the HTTP *status_code* which tells us if the request was successful. HTTP code 200 stands for *Sucess OK*.

In [22]:
import requests

response = requests.get('https://www.spatialthoughts.com')

print(response.status_code)

200


# Little Bit about OpenRouteServices

[OpenRouteService (ORS)](https://openrouteservice.org/) provides a free API for routing, distance matrix, geocoding, route optimization etc. using OpenStreetMap data. We will learn how to use this API through Python and get real-world distance between cities.

In [23]:
ORS_API_KEY = "eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6ImE3YmZkYzRkMGExMzRjNWM4NzAzM2RlMjY4NzdkMTM0IiwiaCI6Im11cm11cjY0In0="

In [24]:
import requests

san_francisco = (37.7749, -122.4194)
new_york = (40.661, -73.944)

parameters = {
    'api_key': ORS_API_KEY,
    'start' : '{},{}'.format(san_francisco[1], san_francisco[0]),
    'end' : '{},{}'.format(new_york[1], new_york[0])
}

response = requests.get(
    'https://api.openrouteservice.org/v2/directions/driving-car', params=parameters)

if response.status_code == 200:
    print('Request successful.')
    data = response.json()
else:
    print('Request failed.')


Request successful.


We can read the `response` in JSON format by calling `json()` method on it.

In [25]:
data = response.json()

In [26]:
data 

{'type': 'FeatureCollection',
 'bbox': [-122.419451, 37.770355, -73.944025, 41.789337],
 'features': [{'bbox': [-122.419451, 37.770355, -73.944025, 41.789337],
   'type': 'Feature',
   'properties': {'segments': [{'distance': 4692967.1,
      'duration': 167127.4,
      'steps': [{'distance': 247.6,
        'duration': 32.5,
        'type': 11,
        'instruction': 'Head northeast on Market Street',
        'name': 'Market Street',
        'way_points': [0, 5]},
       {'distance': 962.6,
        'duration': 120.9,
        'type': 1,
        'instruction': 'Turn right onto 10th Street',
        'name': '10th Street',
        'way_points': [5, 23]},
       {'distance': 382.1,
        'duration': 45.1,
        'type': 0,
        'instruction': 'Turn left onto Bryant Street',
        'name': 'Bryant Street',
        'way_points': [23, 30]},
       {'distance': 18764.8,
        'duration': 1194.2,
        'type': 12,
        'instruction': 'Keep left',
        'name': '-',
        'way_p

The response is a GeoJSON object representing the driving direction between the 2 points. The object is a feature collection with just 1 feature. We can access it using the index **0**. The feature's property contains `summary` information which has the data we need.

In [27]:
summary = data['features'][0]['properties']['summary']
print(summary)

{'distance': 4692967.1, 'duration': 167127.4}


We can extract the `distance` and convert it to kilometers.

In [28]:
distance = summary['distance']
print(distance/1000)

4692.9671


## API Rate Limiting

Many web APIs enforce *rate limiting* - allowing a limited number of requests over time. With computers it is easy to write a for loop, or have multiple programs send hundrends or thousands of queries per second. The server may not be configured to handle such volume. So the providers specify the limits on how many and how fast the queries can be sent.

OpenRouteService lists several [API Restrictions](https://openrouteservice.org/plans/). The free plan allows for upto 40 direction requests/minute.

There are many libraries available to implement various strategies for rate limiting. But we can use the built-in `time` module to implement a very simple rate limiting method.

# Reading CSV Files

Comma-separated Values (CSV) are the most common text-based file format for sharing geospatial data. The structure of the file is 1 data record per line, with individual *columns* separated by a comma. 

In general, the separator character is called a delimiter. Other popular delimiters include the tab (\\t), colon (:) and semi-colon (;) characters. 

Reading CSV file properly requires us to know which delimiter is being used, along with quote character to surround the field values that contain space of the delimiter character. Since reading delimited text file is a very common operation, and can be tricky to handle all the corner cases, Python comes with its own library called `csv` for easy reading and writing of CSV files. To use it, you just have to import it.


In [29]:
import csv 

The preferred way to read CSV files is using the `DictReader()` method. Which directly reads each row and creates a dictionary from it - with column names as *key* and column values as *value*. Let's saee how to read a file using the `csv.DictReader()` method.


In [30]:
import os
data_pkg_path = 'data'
filename = 'worldcities.csv'
path = os.path.join(data_pkg_path, filename)

In [31]:
f = open(path, 'r')
csv_reader = csv.DictReader(f, delimiter=',', quotechar='"')
print(csv_reader)
f.close()

### we can also use the with statement which automatically closes the opened file

## Filtering rows

We can use conditional statement while iterating over the rows, to select and process rows that meet certain criterial. Let's count how many cities from a particular country are present in the file.

Replace the `home_country` variable with your home country below.

In [32]:
home_country = 'India'
num_cities = 0

with open(path, 'r', encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)

    for row in csv_reader:
        if row['country'] == home_country:
            num_cities += 1
            
print(num_cities)

212


## Calculating distance

Let's apply the skills we have learnt so far to solve a complete problem. We want to read the `worldcities.csv` file, find all cities within a home country, calculate the distance to each cities from a home city and write the results to a new CSV file.

First we find the coordinates of the out selected `home_city` from the file. Replace the `home_city` below with your hometown or a large city within your country. Note that we are using the `city_ascii` field for city name comparison, so make sure the `home_city` variable contains the ASCII version of the city name.

In [33]:
home_city = 'Bengaluru'

home_city_coordinates = ()

with open(path, 'r', encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)
    for row in csv_reader:
        if row['city_ascii'] == home_city:
            lat = row['lat']
            lng = row['lng']
            home_city_coordinates = (lat, lng)
            break
        
print(home_city_coordinates)

('12.9700', '77.5600')


Now we can loop through the file, find a city in the chosen home country and call the `geopy.distance.geodesic()` function to calculate the distance. In the code below, we are just computing first 5 matches.

In [34]:
from geopy import distance

counter = 0
with open(path, 'r', encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)
    for row in csv_reader:
        if (row['country'] == home_country and
            row['city_ascii'] != home_city):
            city_coordinates = (row['lat'], row['lng'])
            city_distance = distance.geodesic(
                city_coordinates, home_city_coordinates).km
            print(row['city_ascii'], city_distance)
            counter += 1
            
        if counter == 5:
            break
            

Mumbai 837.1857087990927
Delhi 1738.6388557826444
Kolkata 1552.6378233436667
Chennai 295.34010730466787
Hyderabad 500.04772863048214


## Writing files

Instead of printing the results, let's write the results to a new file. Similar to csv.DictReader(), there is a companion `csv.DictWriter()` method to write files. We create a `csv_writer` object and then write rows to it using the `writerow()` method.

In [35]:
output_dir = 'output'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [36]:
output_filename = 'cities_distance.csv'
output_path = os.path.join(output_dir, output_filename)

with open(output_path, mode='w', newline='', encoding='utf-8') as output_file:
    fieldnames = ['city', 'distance_from_home']
    csv_writer = csv.DictWriter(output_file, fieldnames=fieldnames)
    csv_writer.writeheader()
    
    # Now we read the input file, calculate distance and
    # write a row to the output 
    with open(path, 'r', encoding='utf-8') as f:
        csv_reader = csv.DictReader(f)
        for row in csv_reader:
            if (row['country'] == home_country and
                row['city_ascii'] != home_city):
                city_coordinates = (row['lat'], row['lng'])
                city_distance = distance.geodesic(
                    city_coordinates, home_city_coordinates).km
                csv_writer.writerow(
                    {'city': row['city_ascii'],
                     'distance_from_home': city_distance}
                )

In [37]:
print('Successfully written output file at {}'.format(output_path))

Successfully written output file at output\cities_distance.csv


# Working With Geopandas


GeoPandas extends the Pandas library to enable spatial operations. It provides new data types such as **GeoDataFrame** and **GeoSeries** which are subclasses of Pandas **DataFrame** and **Series** and enables efficient vector data processing in Python.

GeoPandas make use of many other widely used spatial libraries - but it provides an interface similar to Pandas that make it intuitive to use it with spatial analysis. GeoPandas is built on top of the following libraries that allow it to be spatially aware.

- [Shapely](https://shapely.readthedocs.io/en/latest/manual.html) for geometric operations (i.e. buffer, intersections etc.)
- [PyProj](https://pyproj4.github.io/pyproj/stable/index.html) for working with projections
- [Fiona](https://pypi.org/project/Fiona/) for file input and output, which itself is based on the widely used [GDAL/OGR](https://gdal.org/) library

We will carry out a geoprocessing task that shows various features of this library and show how to do geo data processing in Python. The task is to take a roads data layer from OpenStreetMap and compute the total length of National Highways for each district in a state. The problem is described in detail in my [Advanced QGIS](https://courses.spatialthoughts.com/advanced-qgis.html#exercise-find-the-length-of-national-highways-in-a-state) course and show the steps needed to perform this analysis in QGIS. We will replicate this example in Python.



### Before Reading the file a little bit about gpkg file format

A GeoPackage (GPKG) file is a container format for storing geospatial data in a single file, built on top of a SQLite database. It's an open, non-proprietary, platform-independent, and standards-based format developed by the Open Geospatial Consortium (OGC). Essentially, it's a way to store both vector and raster data, along with attributes and metadata, in a single, easily portable file. 

In [38]:
import os
data_pkg_path = 'data'
filename = 'karnataka.gpkg'
path = os.path.join(data_pkg_path, filename)

GeoPandas has a `read_file()` method that is able to open a wide variety of vector datasets, including zip files. Here we will open the GeoPackage `karnataka.gpkg` and read a layer called `karnataka_major_roads`. The result of the read method is a **GeoDataFrame**. 

In [39]:
# import geopandas as gpd 
# roads_gdf = gpd.read_file(path, layer='karnataka_major_roads')
# roads_gdf

In [40]:
import pandas 
import geopandas as gpd 

In [41]:
# !pip show numpy pandas geopandas


In [42]:
# !pip install --upgrade --force-reinstall "numpy>=1.26" "pandas>=2.2" geopandas


In [43]:
import geopandas as gpd 

In [44]:
# !conda install -c conda-forge numpy pandas geopandas


In [45]:
# !pip install geopandas[all]


In [46]:
import geopandas as gpd 

In [47]:
roads_gdf = gpd.read_file(path, layer='karnataka_major_roads')
roads_gdf

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry
0,4354953,5114,secondary,Mahatma Gandhi Road,None,F,30,0,F,F,"MULTILINESTRING ((77.59928 12.97672, 77.5995 1..."
1,8285861,5114,secondary,None,None,B,0,0,F,F,"MULTILINESTRING ((76.65944 12.31809, 76.65904 ..."
2,8285868,5113,primary,Bangalore Nilgiri Road,None,F,0,0,F,F,"MULTILINESTRING ((76.65906 12.31389, 76.65912 ..."
3,8285890,5115,tertiary,Sri Harsha Road,None,F,0,0,F,F,"MULTILINESTRING ((76.656 12.30895, 76.65646 12..."
4,8285892,5114,secondary,Ashoka Road,None,F,0,0,F,F,"MULTILINESTRING ((76.65615 12.30989, 76.656 12..."
...,...,...,...,...,...,...,...,...,...,...,...
44601,763293423,5114,secondary,Kamaraj Road,None,F,0,0,F,F,"MULTILINESTRING ((77.60806 12.97517, 77.60797 ..."
44602,763293424,5114,secondary,Mahatma Gandhi Road,None,F,30,0,F,F,"MULTILINESTRING ((77.60798 12.97519, 77.60806 ..."
44603,763295978,5114,secondary,18th Cross Road,None,B,0,0,F,F,"MULTILINESTRING ((77.57115 13.00849, 77.57156 ..."
44604,763405052,5113,primary,Varthur Road,None,F,0,0,F,F,"MULTILINESTRING ((77.7014 12.95693, 77.70164 1..."


A GeoDataFrame contains a special column called *geometry*. All spatial operations on the GeoDataFrame are applied to the geomety column. The geometry column can be accessed using the `geometry` attribute.

In [48]:
roads_gdf.geometry

0        MULTILINESTRING ((77.59928 12.97672, 77.5995 1...
1        MULTILINESTRING ((76.65944 12.31809, 76.65904 ...
2        MULTILINESTRING ((76.65906 12.31389, 76.65912 ...
3        MULTILINESTRING ((76.656 12.30895, 76.65646 12...
4        MULTILINESTRING ((76.65615 12.30989, 76.656 12...
                               ...                        
44601    MULTILINESTRING ((77.60806 12.97517, 77.60797 ...
44602    MULTILINESTRING ((77.60798 12.97519, 77.60806 ...
44603    MULTILINESTRING ((77.57115 13.00849, 77.57156 ...
44604    MULTILINESTRING ((77.7014 12.95693, 77.70164 1...
44605    MULTILINESTRING ((77.6104 12.9736, 77.61052 12...
Name: geometry, Length: 44606, dtype: geometry

### Filtering Data

In [49]:
filtered = roads_gdf[roads_gdf['ref'].str.match('^NH') == True]
filtered

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry
17,8684837,5112,trunk,Bengaluru - Mangaluru Road,NH373,F,0,0,F,F,"MULTILINESTRING ((76.10024 13.00326, 76.0995 1..."
26,9951034,5112,trunk,None,NH948,B,50,0,F,F,"MULTILINESTRING ((77.16472 12.24774, 77.16416 ..."
54,22838314,5112,trunk,Solapur-Mangalore Highway,NH169,B,0,0,F,F,"MULTILINESTRING ((74.86387 12.88387, 74.86418 ..."
55,22838318,5112,trunk,None,NH66,B,0,0,F,F,"MULTILINESTRING ((74.78756 13.09142, 74.78744 ..."
56,22838318,5112,trunk,None,NH66,B,0,0,F,F,"MULTILINESTRING ((74.78767 13.09723, 74.78767 ..."
...,...,...,...,...,...,...,...,...,...,...,...
44466,760685447,5112,trunk,Outer Ring Road,NH75,F,60,0,F,F,"MULTILINESTRING ((77.5785 13.04769, 77.57867 1..."
44531,762411542,5112,trunk,None,NH67,B,0,0,F,F,"MULTILINESTRING ((76.45075 15.24065, 76.45024 ..."
44550,762500475,5112,trunk,Bangalore-Mysore Road,NH275,F,60,0,F,F,"MULTILINESTRING ((77.51843 12.93633, 77.51825 ..."
44551,762500476,5112,trunk,Mysore Road,NH275,F,0,0,F,F,"MULTILINESTRING ((77.53309 12.94741, 77.53295 ..."


## Working with Projections

Dealing with projetions is a key aspect of working with spatial data. GeoPandas uses the `pyproj` library to assign and manage projections. Each GeoDataFrame as a `crs` attribute that contains the projection info. Our source dataset is in EPSG:4326 WGS84 CRS.

In [50]:
filtered.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Since our task is to compute line lengths, we need to use a Projected CRS. We can use the `to_crs()` method to reproject the GeoDataFrame.

In [51]:
roads_reprojected = filtered.to_crs('EPSG:32643')
roads_reprojected.crs

<Projected CRS: EPSG:32643>
Name: WGS 84 / UTM zone 43N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 72°E and 78°E, northern hemisphere between equator and 84°N, onshore and offshore. China. India. Kazakhstan. Kyrgyzstan. Maldives. Pakistan. Russian Federation. Tajikistan.
- bounds: (72.0, 0.0, 78.0, 84.0)
Coordinate Operation:
- name: UTM zone 43N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Now that the layer has been reprojected, we can calculate the length of each geometry using the `length` attribute. The result would be in meters. We can add the line lengths in a new column named `length`.

In [52]:
roads_reprojected['length'] = roads_reprojected['geometry'].length

We can apply statistical operations on a DataFrame columns. Here we can compute the total length of national highways in the state by calling the `sum()` method.

In [53]:
total_length = roads_reprojected['length'].sum()
print('Total length of national highways in the state is {} KM'.format(total_length/1000))

Total length of national highways in the state is 8616.166058402969 KM


## Performing Spatial joins

There are two ways to combine datasets in geopandas – table joins and spatial joins. For our task, we need information about which district each road segments belongs to. This can be achived using another spatial layer for the districts and doing a spatial join to transfer the attributes of the district layer to the matching road segment.

The `karnataka.gpkg` contains a layer called `karnataka_districts` with the district boundaries and names.

In [55]:
districts_gdf = gpd.read_file(path, layer='karnataka_districts')
districts_gdf

,DISTRICT,ST_NM,ST_CEN_CD,DT_CEN_CD,censuscode,geometry
0,Bagalkot,Karnataka,29,2,556,"MULTIPOLYGON (((76.241 16.16531, 76.23538 16.1..."
1,Bangalore Rural,Karnataka,29,29,583,"MULTIPOLYGON (((77.38701 13.50002, 77.40099 13..."
2,Bangalore,Karnataka,29,18,572,"MULTIPOLYGON (((77.83549 12.86809, 77.83213 12..."
3,Belgaum,Karnataka,29,1,555,"MULTIPOLYGON (((75.02647 16.93264, 75.02827 16..."
4,Bellary,Karnataka,29,11,565,"MULTIPOLYGON (((77.15757 15.13706, 77.15887 15..."
5,Bidar,Karnataka,29,4,558,"MULTIPOLYGON (((77.34032 18.43689, 77.3451 18...."
6,Bijapur,Karnataka,29,3,557,"MULTIPOLYGON (((76.07344 17.33403, 76.0703 17...."
7,Chamrajnagar,Karnataka,29,24,578,"MULTIPOLYGON (((77.32282 12.30709, 77.32726 12..."
8,Chikkaballapura,Karnataka,29,28,582,"MULTIPOLYGON (((78.1996 13.56615, 78.19897 13...."
9,Chikmagalur,Karnataka,29,16,570,"MULTIPOLYGON (((76.35411 13.5777, 76.35611 13...."


Before joining this layer to the roads, we must reproject it to match the CRS of the roads layer.

In [56]:
districts_reprojected = districts_gdf.to_crs('EPSG:32643')

A spatial join is performed using the `sjoin()` method. It takes 2 core arguments.

- `predicate`: The spatial predicate to decdie which objects to join. Options are *intersects*, *within* and *contains*.
- `how`: The type of join to perform. Options are *left*, *right* and *inner*.

For our task, we can do a *left* join and add attributes of the district that *intersect* the road.


In [57]:
joined = gpd.sjoin(roads_reprojected, districts_reprojected, how='left', predicate='intersects')

Note: In this example, some road segments cross polygon boundaries. A spatial join will duplicate these segments for each polygon they intersect, resulting in an overestimation of the total length. A more accurate method is to use a [Spatial Overlay]((https://geopandas.org/en/stable/docs/reference/api/geopandas.overlay.html)), which splits segments at polygon boundaries. The code example below demonstrates this approach.

```
joined = gpd.overlay(roads_reprojected, districts_reprojected, 
                     how='intersection', keep_geom_type=True)
# Update the length of each segment after the overlay
joined['length'] = joined.geometry.length
```

## Group Statistics

In [58]:
results = joined.groupby('DISTRICT')['length'].sum()/1000
results

DISTRICT
Bagalkot            258.449475
Bangalore           314.170539
Bangalore Rural     319.965959
Belgaum             528.922541
Bellary             304.789522
Bidar               247.348300
Bijapur             424.197281
Chamrajnagar        217.737255
Chikkaballapura     211.957819
Chikmagalur         334.423573
Chitradurga         531.932443
Dakshina Kannada    316.339566
Davanagere          177.877407
Dharwad             314.978752
Gadag               111.548768
Gulbarga            294.551578
Hassan              436.231356
Haveri              306.582730
Kodagu               63.806864
Kolar               221.598406
Koppal              288.358711
Mandya              337.555904
Mysore              361.510005
Raichur             167.746669
Ramanagara          199.426362
Shimoga             479.140995
Tumkur              613.177564
Udupi               277.331577
Uttara Kannada      424.184040
Yadgir              144.858328
Name: length, dtype: float64

The result of the `group_by()` method is a Pandas *Series*. It can be saved to a CSV file using the `to_csv()` method.

In [60]:
output_filename = 'national_highways_by_districts.csv'
output_dir = 'output'
output_path = os.path.join(output_dir, output_filename)
results.to_csv(output_path)
print('Successfully written output file at {}'.format(output_path))

Successfully written output file at output\national_highways_by_districts.csv


# Creating Spatial Data

A common operation in spatial analysis is to take non-spatial data, such as CSV files, and creating a spatial dataset from it using coordinate information contained in the file. GeoPandas provides a convenient way to take data from a delimited-text file, create geometry and write the results as a spatial dataset.

We will read a tab-delimited file of places, filter it to a feature class, create a GeoDataFrame and export it as a GeoPackage file.

![](https://github.com/spatialthoughts/python-foundation-web/blob/master/images/python_foundation/geonames_mountains.png?raw=1)

In [61]:
import os
import pandas as pd
import geopandas as gpd

In [62]:
data_pkg_path = 'data/geonames/'
filename = 'US.txt'
path = os.path.join(data_pkg_path, filename)

## Reading Tab-Delimited Files

The source data comes from [GeoNames](https://en.wikipedia.org/wiki/GeoNames) - a free and open database of geographic names of the world. It is a huge database containing millions of records per country. The data is distributed as country-level text files in a tab-delimited format. The files do not contain a header row with column names, so we need to specify them when reading the data. The data format is described in detail on the [Data Export](https://www.geonames.org/export/) page.

We specify the separator as **\\t** (tab) as an argument to the `read_csv()` method. Note that the file for USA has more than 2M records.

In [64]:
column_names = [
    'geonameid', 'name', 'asciiname', 'alternatenames',
    'latitude', 'longitude', 'feature class', 'feature code',
    'country code', 'cc2', 'admin1 code', 'admin2 code',
    'admin3 code', 'admin4 code', 'population', 'elevation',
    'dem', 'timezone', 'modification date'
]

df = pd.read_csv(path, sep='\t', names=column_names)
df.info()

C:\Users\Rudra Sai Praveen\AppData\Local\Temp\ipykernel_16900\3259660362.py:9: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep='\t', names=column_names)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2237861 entries, 0 to 2237860
Data columns (total 19 columns):
 #   Column             Dtype  
---  ------             -----  
 0   geonameid          int64  
 1   name               object 
 2   asciiname          object 
 3   alternatenames     object 
 4   latitude           float64
 5   longitude          float64
 6   feature class      object 
 7   feature code       object 
 8   country code       object 
 9   cc2                object 
 10  admin1 code        object 
 11  admin2 code        object 
 12  admin3 code        float64
 13  admin4 code        float64
 14  population         int64  
 15  elevation          float64
 16  dem                int64  
 17  timezone           object 
 18  modification date  object 
dtypes: float64(5), int64(3), object(11)
memory usage: 324.4+ MB


## Filtering Data

In [65]:
mountains = df[df['feature class']=='T']
mountains.head()[['name', 'latitude', 'longitude', 'dem','feature class']]

,name,latitude,longitude,dem,feature class
15,Vulcan Point,52.10222,177.53889,-9999,T
16,Tropical Ridge,51.99167,177.50833,267,T
17,Thirty-Seven Hill,52.84528,173.15278,193,T
20,Square Point,52.86120,173.33679,30,T
21,Square Bluff,51.65000,178.70000,-9999,T


## Creating Geometries

GeoPandas has a conveinent function `points_from_xy()` that creates a Geometry column from X and Y coordinates. We can then take a Pandas dataframe and create a GeoDataFrame by specifying a CRS and the geometry column.

In [66]:
geometry = gpd.points_from_xy(mountains.longitude, mountains.latitude)
gdf = gpd.GeoDataFrame(mountains, crs='EPSG:4326', geometry=geometry)
gdf

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date,geometry
15,4045410,Vulcan Point,Vulcan Point,"Volcan Point,Vulcan Point",52.10222,177.53889,T,CAPE,US,NaN,AK,16.0,NaN,NaN,0,NaN,-9999,America/Adak,2014-10-08,POINT (177.53889 52.10222)
16,4045411,Tropical Ridge,Tropical Ridge,NaN,51.99167,177.50833,T,RDGE,US,NaN,AK,16.0,NaN,NaN,0,NaN,267,America/Adak,2010-01-30,POINT (177.50833 51.99167)
17,4045412,Thirty-Seven Hill,Thirty-Seven Hill,NaN,52.84528,173.15278,T,MT,US,NaN,AK,16.0,NaN,NaN,0,NaN,193,America/Adak,2014-10-08,POINT (173.15278 52.84528)
20,4045415,Square Point,Square Point,NaN,52.86120,173.33679,T,CAPE,US,NaN,AK,16.0,NaN,NaN,0,NaN,30,America/Adak,2014-11-17,POINT (173.33679 52.8612)
21,4045416,Square Bluff,Square Bluff,NaN,51.65000,178.70000,T,CLF,US,NaN,AK,16.0,NaN,NaN,0,NaN,-9999,America/Adak,2014-10-08,POINT (178.7 51.65)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2237736,12123289,Alum Cave Bluff,Alum Cave Bluff,NaN,35.63888,-83.44603,T,CLF,US,NaN,TN,155.0,NaN,NaN,0,1510.0,1470,America/New_York,2020-02-04,POINT (-83.44603 35.63888)
2237746,12123685,Arch Rock,Arch Rock,NaN,35.63241,-83.44496,T,RK,US,NaN,TN,155.0,NaN,NaN,0,1338.0,1206,America/New_York,2020-02-05,POINT (-83.44496 35.63241)
2237763,12129575,Hole-in-the-Rock,Hole-in-the-Rock,NaN,37.25431,-110.89622,T,RDGE,US,NaN,UT,25.0,NaN,NaN,0,NaN,1219,America/Denver,2020-03-03,POINT (-110.89622 37.25431)
2237780,12148792,Fish Lake Valley,Fish Lake Valley,NaN,37.38330,-117.83330,T,VAL,US,NaN,CA,27.0,NaN,NaN,0,NaN,1627,America/Los_Angeles,2020-03-21,POINT (-117.8333 37.3833)


## Writing Files

We can write the resulting GeoDataFrame to any of the supported vector data format. The format is inferred from the file extension. Use `.shp` if you want to save the results as a shapefile. Here we are writing it as a new GeoPackage file so we use the `.gpkg` extension.

You can open the resulting geopackage in a GIS and view the data.

In [67]:
output_dir = 'output'
output_filename = 'mountains.gpkg'
output_path = os.path.join(output_dir, output_filename)

gdf.to_file(filename=output_path, encoding='utf-8')
print('Successfully written output file at {}'.format(output_path))

Successfully written output file at output\mountains.gpkg



# Working with RasterIO

[RasterIO](https://rasterio.readthedocs.io/en/latest/) is a modern library to work with geospatial data in a gridded format. It excels at providing an easy way to read/write raster data and access individual bands and pixels as `numpy` arrays.

RasterIO is built on top of the popular [GDAL (Geospatial Data Abstraction Library)](https://gdal.org/). GDAL is written in C++ so the Python API provided by GDAL is not very intuitive for Python users. RaserIO aims to make it easy for Python users to use the underlying GDAL library in an intuitive way.

In [70]:
import rasterio 

In [71]:
import os
data_pkg_path = 'data'
srtm_dir = 'srtm'
filename = 'N28E087.hgt'
path = os.path.join(data_pkg_path, srtm_dir, filename)

## Reading Raster Data

RasterIO can read any raster format supported by the GDAL library. We can call the `open()` method with the file path of the raster. The resulting dataset behaves much like Python's File object.

In [72]:
dataset = rasterio.open(path)

You can check information about the raster using the `meta` attribute.

An important property is the dataset *transform*. The transform contains the pixel resolution of the dataset and the row and column coordinates of the upper left corner of the dataset.

In [73]:
metadata = dataset.meta
metadata

{'driver': 'SRTMHGT',
 'dtype': 'int16',
 'nodata': -32768.0,
 'width': 3601,
 'height': 3601,
 'count': 1,
 'crs': CRS.from_wkt('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'),
 'transform': Affine(0.0002777777777777778, 0.0, 86.99986111111112,
        0.0, -0.0002777777777777778, 29.000138888888888)}

To read the pixel values, we need to call the `read()` method by passing it a band’s index number. Following the GDAL convention, bands are indexed from 1. Since our dataset contain just 1-band, we can read it as follows.

In [74]:
band1 = dataset.read(1)
band1

array([[5217, 5211, 5208, ..., 5097, 5098, 5089],
       [5206, 5201, 5200, ..., 5080, 5075, 5069],
       [5199, 5194, 5191, ..., 5063, 5055, 5048],
       ...,
       [5347, 5345, 5343, ..., 5747, 5750, 5757],
       [5338, 5338, 5336, ..., 5737, 5740, 5747],
       [5332, 5331, 5332, ..., 5734, 5736, 5744]], dtype=int16)

Finally, when we are done with the dataset, we must close it. It is especially important when writing a dataset.block_shapes

In [75]:
dataset.close()

## Merging Datasets

Let's see how we can read the 4 individual tiles and mosaic them together. RasterIO provides multiple sub-modules for various raster operations. We can use the `rasterio.merge` module to carry out this operation.

In [76]:
srtm_path = os.path.join(data_pkg_path, 'srtm')
all_files = os.listdir(srtm_path)
all_files

['N27E086.hgt', 'N27E087.hgt', 'N28E086.hgt', 'N28E087.hgt']

The rasterio.merge module has a `merge()` method that takes a list of *datasets* and returns the merged dataset. So we create an empty list, open each of the files and append it to the list.

In [77]:
dataset_list = []
for file in all_files:
    path = os.path.join(srtm_path, file)
    dataset_list.append(rasterio.open(path))
dataset_list

[<open DatasetReader name='data\srtm\N27E086.hgt' mode='r'>,
 <open DatasetReader name='data\srtm\N27E087.hgt' mode='r'>,
 <open DatasetReader name='data\srtm\N28E086.hgt' mode='r'>,
 <open DatasetReader name='data\srtm\N28E087.hgt' mode='r'>]

We can pass on the list of tile dataset to the merge method, which will return us the merged data and a new *transform* which contains the updated extent of the merged raster.

In [79]:
from rasterio import merge
merged_result = merge.merge(dataset_list)
merged_result

(array([[[4916, 4926, 4931, ..., 5097, 5098, 5089],
         [4919, 4932, 4928, ..., 5080, 5075, 5069],
         [4919, 4928, 4935, ..., 5063, 5055, 5048],
         ...,
         [ 368,  368,  366, ..., 1905, 1919, 1937],
         [ 364,  364,  362, ..., 1913, 1930, 1944],
         [ 360,  359,  357, ..., 1918, 1930, 1942]]], dtype=int16),
 Affine(0.0002777777777777778, 0.0, 85.99986111111112,
        0.0, -0.0002777777777777778, 29.000138888888888))

In [80]:
merged_data = merged_result[0]
merged_transform = merged_result[1]

Verify that the resulting array shape the sum of individual rasters

In [82]:
merged_data.shape

(1, 7201, 7201)

## Writing Raster Data

Similar to regular Python files, to create a new file, we can open the output file in the *write* mode. RasterIO provides a `write()` method that we can use to write individual bands.

In [83]:
output_filename = 'merged.tif'
output_dir = 'output'
output_path = os.path.join(output_dir, output_filename)

We need to specify many metadata parameters to initialize the output dataset. Some of these parameter values can be directly copied from the input files, such as `crs`, `dtype`, `nodata` etc. , while others can be obtained from the merged dataset, such as `height` and `width`.

Remember to call the `close()` method which will finalize the file and write the data to disk.

In [84]:
new_dataset = rasterio.open(output_path, 'w', 
                            driver='GTiff',
                            height=merged_data.shape[1],
                            width=merged_data.shape[2],
                            count=1,
                            nodata=-32768.0,
                            dtype=merged_data.dtype,
                            crs='EPSG:4326',
                            transform=merged_transform)
new_dataset.write(merged_data)
new_dataset.close()
print('Successfully written output file at {}'.format(output_path))

Successfully written output file at output\merged.tif
